# Generative Adversarial Networks

Les modèles antagonistes génératifs ou [*Generative Adversarial Networks* (GAN)](https://arxiv.org/pdf/1406.2661) sont une famille de modèles génératifs. De manière assez évidente, les modèles génératifs sont des modèles non supervisés, c'est-à-dire, que l'on ne dispose pas de labels pendant l'entraînement (on peut avoir des labels mais ceux-ci ne constitue pas notre objectif d'entraînement). Les GAN introduisent une astuce pour transformer ce problème non supervisé en un problème supervisé.   

## Principe de base

### Deux modèles concurrents

Le principe de base derrière l'architecture GAN est d'utiliser deux réseaux de neurones que l'on va entraîner de manière concurrente.   
Les deux modèles sont les suivants : 
- **Le modèle générateur** : Ce modèle a pour objectif de générer des exemples qui sont similaires aux exemples des données d'entraînement. Il prend en entrée un vecteur aléatoire (issu d'une distribution gaussienne) de taille fixe et génére un exemple à partir de cette entrée. 
- **Le modèle discriminateur** : Ce modèle a pour objectif de classifier un exemple comme étant réel (issu des données réelles d'entrainement) ou faux (généré par le modèle générateur). C'est un simple classifier comme ceux que l'on a pu voir dans les cours précédents.

<img src="images/GAN.png" alt="GAN" width="800"/>

Figure extraite de [blogpost](https://towardsdatascience.com/generative-adversarial-networks-explained-34472718707a).

### Zero-sum game

Ces modèles sont entrainés de manière concurrentes dans ce qu'on appelle un *zero-sum game*. Notre objectif d'entraînement est que le modèle discriminatif soit incapables de déterminer si un exemple est réel ou généré (il nous renvoie la probabilité $\frac{1}{2}$ sur chaque élément). Cela signifiera que notre générateur est capable de générer des exemples plausibles. Dans ce cas là, le modèle générateur a appris la distribution de probabilité des données d'entraînement.

Lors de l'entraînement, le générateur va générer un *batch* d'exemples que l'on va mélanger avec des exemples réels (issus des données d'entrainement). Ces exemples sont donnés aux discriminateur qui va les classifier.  
Le discriminateur est ensuite mis à jour par descente du gradient en fonction de sa performance sur la classification.   
Le générateur est ensuite mis à jour en fonction de sa performance de tromperie du discriminateur. 

On peut faire une analogie où le générateur serait un faussaire et le discriminateur serait la police. Le but du faussaire est de tromper la police en générant des faux billets le plus parfait possible. A l'inverse, la police va créer de nouvelles techniques pour déterminer si les billets sont vrais ou faux. De cette manière, le faussaire et la police vont progresser conjointement.  

Je trouve personnellement que l'idée derrière les GAN peut s'appliquer à beaucoup de situations de la vie réelle. On a tendance à donner le meilleur de nous-même dans l'adversité.

**Note** : En général, dans l'entrainement d'un GAN, on est interessé principalement par le générateur. Le discriminateur n'est alors utilisé que pour l'entrainement. Cependant, comme la quantité d'images pour un entraînement non supervisé est beaucoup plus importante que les images avec labels (les images non supervisées sont en gros l'ensemble des images disponibles sur internet), il est parfois intéressant d'entraîner un GAN sur une grande quantité d'images et d'utiliser le discriminateur et le générateur comme modèles pré-entrainé que l'on va adapter à d'autres tâches.

### Architecture du GAN

La grande majorité des GAN sont des réseaux de neurones convolutifs et ce pour plusieurs raisons. D'une part, il est assez facile pour un utilisateur humain d'évaluer la qualité d'une image générée, ce qui permet une évaluation directe et intuitive des performances du modèle. D'autre part, empiriquement, les GAN ont demontrés des performances impressionnantes pour tout ce qui est génération d'image mais moins pour des tâches différentes comme la génération de texte. 

L'architecture de la plupart des GAN se base sur le papier [DCGAN](https://arxiv.org/pdf/1511.06434).

**Note** : L'invention des GAN est antérieure à l'invention de l'architecture transformer. Récemment des architectures type GAN ont été proposées avec un transformer à la place du CNN.

**Note 2** : Il y a un aspect théorique important derrière les GAN mais nous n'allons pas entrer dans les détails dans ce cours. Pour en apprendre plus, vous pouvez vous réferrer au cours CS236 de stanford et en particulier à ce [lien](https://deepgenerativemodels.github.io/notes/gan/).

## Conditional GAN

L'architecture que nous avons vu permet de générer une image réaliste à partir d'un vecteur aléatoire sample depuis une distribution gaussienne. Cependant, nous n'avons aucun contrôle sur l'image générée. Si on entrainé le modèle sur des images de personnes, le modèle va générer une personne avec des attributs aléatoire (sexe, yeux, cheveux, peau etc...). Cela n'est pas très pratique car dans de nombreux cas, on cherche à générer une image spécifique et pas seulement une image réaliste aléatoire. 

Pour palier à ce problème, on peut utiliser ce qu'on appelle un *conditional GAN* qui reprend l'architecture classique du GAN mais dans lequel on donne en entrée du générateur et du discriminateur une information sur la données. 

<img src="images/cGAN.png" alt="GAN" width="800"/>


Figure extraite du [blogpost](https://towardsdatascience.com/cgan-conditional-generative-adversarial-network-how-to-gain-control-over-gan-outputs-b30620bd0cc8)

De cette manière, on peut diriger la génération à l'aide d'un label ce qui va nous permettre de générer des images contenant des attributs spécifiques.



## Problèmes des GAN

L'architecture GAN est une très bonne idée et fonctionne très bien en pratique lorsque le modèle est bien entrainé. Dans les cours précédents, nous avons insisté sur la difficulté à entraîner un modèle de deep learning et nous avons présenté de nombreuses techniques pour faciliter l'entraînement. Ici, nous avons deux modèles à entraîner en même temps et de manière adversaire... C'est ça le principal problème des GAN : En pratique, ils sont très compliqué à entraîner.

### Mode Collapse

Le problème principal qui fait frissonner tous les utilisateurs de GAN est le *mode collapse*. Cela se produit quand le générateur va apprendre à produire une variété limitée de résultats souvent très similaires les uns aux autres. Dans ce cas là, le générateur n'a pas réussi à capturer la diversité des données d'entraînement mais parvient quand même à duper le discriminateur systématiquement. On peut imaginer que le générateur a appris à générer une image parfaite mais qu'il ne peut générer que cette image.  

Ce problème découle directement de l'objectif d'entrainement du GAN et il est très difficile à gérer. 

### Equilibre entre générateur et discriminateur

Lors de l'entraînement, on souhaite que le générateur et le discriminateur progresse conjointement. Cependant, il peut arriver qu'un des deux modèles progresse plus rapidement que l'autre ce qui peut causer des comportements chaotiques lors de l'entraînement.

Il y a également d'autres problèmes que l'on peut citer rapidement : 
- Problème de convergence : Le modèle peut du mal à converger vers une solution stable même après un long entraînement.
- Choix de l'architecture des réseaux : Il faut choisir une architecture cohérente pour à la fois le générateur et le discriminateur.

Quelques stratégies sont utilisables pour limiter ces problèmes mais ce n'est rien de magique. Pour stabiliser l'entraînement, on peut utiliser un [Wasserstein GAN](https://arxiv.org/pdf/1701.07875) ou/et un entraînement plus progressif.

## Exemples d'utilisation des GAN

### GAN pour la super résolution

Il est possible d'utiliser les GAN pour faire de la super-résolution, c'est-à-dire d'augmenter la résolution d'une image. Le papier [Photo-Realistic Single Image Super-Resolution Using a Generative Adversarial
Network](https://arxiv.org/pdf/1609.04802) propose une architecture de GAN permettant cela.  

<img src="images/srGAN.png" alt="srGAN" width="800"/>


### GAN pour la data augmentation

La data-augmentation consiste à augmenter artificillement les données d'entraînement via diverses techniques. Si ce n'est pas déjà fait, vous pouvez faire [le cours bonus sur la data augmentation](../Bonus_CoursSpécifiques/03_DataAugmentation.ipynb). 

En y réfléchissant, la data augmentation de base (crop, rotate etc ...) peut être vue comme un sorte de *generative modeling* où l'on va générer des images d'entraînement proches de la distribution des images normales. 

En partant de cette considération, il est assez évident de voir comme un GAN peut nous aider pour la data augmentation. Si on veut classifier les chats, notre GAN va pouvoir nous générer des images de chats en quantité. 

### Image-to-Image translation

Une autre utilisation commune des GAN est le *Image-to-Image translation* introduit dans le papier [Image-to-Image Translation with Conditional Adversarial Networks](https://arxiv.org/pdf/1611.07004). Cela permet de convertir une image d'un domaine source vers un domaine cible tout en gardant les correspondances structurelles et contextuelles (en fonction de l'entraînement).

<img src="images/Im2Im.png" alt="Im2im" width="800"/>


**Note** : On peut aussi utiliser le *Image-to-Image translation* pour la data augmentation. Supposons que l'on génére des fausses données à l'aide d'un logiciel de jeux vidéos comme Unity et que l'on souhaite rendre ces images réalistes. Alors il suffit d'entraîner un GAN de transfert de style permettant de passer d'une image à une autre. 